Power Method
-------------

In [1]:
import numpy as np
import xarray as xr


ds_1 = xr.open_dataset("data/regions_verify_isotope_202112_cov.nc")
bio_1 = ds_1["covariance_bio"]
anth_1 = ds_1["covariance_anth"]

ds_2 = xr.open_dataset("data/regions_verify_202104_cov.nc")
bio_2 = ds_2["covariance_bio"]
anth_2 = ds_2["covariance_anth"]


Functions that compute:
* the approximative, normalized eigenvector corresponding to the eigenvalue of largest magnitude (function from Wikipedia)
* the eigenvalue corresponding to given, normalized eigenvector (my own code)
* diagnostics to see how well the power method performs (my own code)

In [2]:
#!/usr/bin/env python3

import numpy as np

def power_iteration(A, num_simulations: int):
    #From: https://en.wikipedia.org/wiki/Power_iteration
    # Retrieved: 8.9.2022
    # Ideally choose a random vector
    # To decrease the chance that our vector
    # Is orthogonal to the eigenvector
    b_k = np.random.rand(A.shape[1])

    for _ in range(num_simulations):
        # calculate the matrix-by-vector product Ab
        b_k1 = np.dot(A, b_k)

        # calculate the norm
        b_k1_norm = np.linalg.norm(b_k1)

        # re normalize the vector
        b_k = b_k1 / b_k1_norm

    return b_k

def rayleigh_quotient(A, v):
    """Function computes the rayleigh quotient for 2-d numpy array A and normalized vector v.
    (Because v normalized, the denominator v.T.dot(v)=1, and can thus be omitted.)
    If v is an eigenvector of A, the quotient gives the corresponding eigenvalue."""
    a = v.T.dot(A).dot(v)
    return a

def residual_norm(A, v, a):
    """Compute the 2-norm of the residual vector r = Av-av, where
    A is a 2-d numpy array, v an approximated eigenvector and a is an approximation of
    corresponding eigenvalue. Can be used to evaluate the accuracy of the estimated
    eigenvector and -value."""
    r = A.dot(v)-a*v
    norm = np.linalg.norm(r)
    return norm

Hotelling's deflation
-----------------------
A method for finding the next eigenvectors after the first one 

In [3]:

def hotelling(A, v, a, num_simulations):
    """GIVES WRONG RESULT FOR SOME REASON
    Function to compute the following eigenvector after the
    one with largest corresponding eigenvalue has been computed.
    Function utilizes Hotelling's method.
    Parameters:
    A : 2D array
    v : 1D array. Eigenvector corresponding to the largest eigenvalue of A. 
    a : float. Largest eigenvalue of A.
    num_simulations : int"""
    b_k = np.random.rand(A.shape[1])

    for i in range(num_simulations):
        b_k1 = A.dot(b_k)-a*v.dot(v.T.dot(b_k))

        norm = np.linalg.norm(b_k1)

        #normalize
        b_k = b_k1 / norm

    return b_k






In [12]:
M = np.diag([1,2,3,4,5])

v = power_iteration(M, 50)
a = rayleigh_quotient(M,v)
v_2 = hotelling(M,v,a,50)

print(v_2)

[ 3.86778237e-33  1.01667941e-15  6.81441966e-07  1.00000000e+00
 -2.92940314e-06]


In [10]:
#test if proj and perp work as they should

v = np.array([3,-1,2])
w1 = np.array([1,1,0])
w2 = np.array([-1,1,1])
ws = [w1, w2]

perp(v,ws)



array([ 1.33333333, -1.33333333,  2.66666667])

In [ ]:
def proj(v,w):
    """Compute the projection of v onto span(w).
    Parameters: v : list or 1-d numpy array
                w : list or 1-d numpy array"""
    p = v.dot(w)/(w.dot(w))*w
    return p

def perp(v, ws):
    """Compute the kohtisuora komponentti of v against subspace span(ws)
    Parameters: v : list or 1-d numpy array
                ws : a list of lists or 1-d numpy arrays. These have to
                be orthogonal"""
    projections = [proj(v,w) for w in ws]
    return v-np.sum(projections, axis=0)


Below the "real" eigenvalues and eigenvectors
----------------------------------------------

In [13]:


#compute eigenvalues and eigenvectors of M. Note: np.linalg.eigh gives eigenvalues in ascending order, 
# so the largest is last, and so is the corresponding eigenvector. 
# (NOTE: Don't need to worry abot how the order treats negative values as all eigenvalues of a positive semidefinite matrix are nonnegative)

evals, evecs = np.linalg.eigh(M) 
#largest eigenvalue and corresponding evec
eval1, evec1 = evals[-1], evecs[-1]
#second largest
eval2, evec2 = evals[-2], evecs[-2]

print(v_2)
print(eval1)
print(eval2)
print(evec2[:])


[ 3.86778237e-33  1.01667941e-15  6.81441966e-07  1.00000000e+00
 -2.92940314e-06]
5.0
4.0
[0. 0. 0. 1. 0.]


In [7]:
for i in np.arange(len(evec1)):
    x = evec1[i]
    if x != 0.0:
        print(i,x)

11 1.0


TypeError: dot only operates on DataArrays.

In [16]:
#print(f"Real eigval: {eval1}")


for i in range(0,50, 5):
    v = power_iteration(M, i)
    a = rayleigh_quotient(M, v)
    norm = residual_norm(M, v, a)
    print(f"residual norm: {norm}")
    #print(f"Iteration: {i}, estimated eigval: {a}, residual norm: {norm}")
    
    

residual norm: 642006.4674358666
residual norm: 2.0196493069835415
residual norm: 0.5447581433695631
residual norm: 0.11979459428156726
residual norm: 0.034124300627580824
residual norm: 0.008680534135338232
residual norm: 0.0029183860978654543
residual norm: 0.0007452193817419561
residual norm: 0.0001947527931012749
residual norm: 6.003346070176393e-05
